In [1]:
from jedi.plugins.stdlib import collections_namedtuple
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

In [2]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY")

print(api_key)

sk-b0e222d65d2c444ca02617cf20fbd136


In [3]:
from glob import glob

text_lines = []

for file_path in glob("faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [4]:
len(text_lines)

72

In [5]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.ai/v1",
)


In [6]:
from pymilvus import model as milvus_model

#print(milvus_model.DefaultEmbeddingFunction().encode_queries(["This is a test"])[0])

embedding_model = milvus_model.DefaultEmbeddingFunction()

/Users/jackma/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [9]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])


NameError: name 'embedding_model' is not defined

In [1]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

完全无法连接，请尝试以下方案


In [2]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collections"

In [ ]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [3]:
milvus_client.create_collection(
    collections_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_type="Strong",
)

NameError: name 'embedding_dim' is not defined

In [ ]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector":doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

In [ ]:
question = "How is data stored in milvus?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question]),
    limit=3,
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"],
)

In [ ]:
import json

retrieved_lines_with_distance = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

print(json.dumps(retrieved_lines_with_distance, indent=4))

In [ ]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distance]
)

In [ ]:
question

In [ ]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [ ]:
USER_PROMPT

In [ ]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "text": SYSTEM_PROMPT},
        {"role": "user", "text": USER_PROMPT},
    ]
)

print(response.choices[0].message.content)